## build the cpp project

make sure the boost library installation directory in the CMakeLists.txt, and
```bash
mkdir build && cd build
cmake ..
make
```

In [1]:
using LinearAlgebra, SkewLinearAlgebra
using GenericTensorNetworks, Graphs
import Nemo
using Scanf


Welcome to Nemo version 0.35.3

Nemo comes with absolutely no warranty whatsoever



In [2]:
function ordered(a, b)
        if a > b
            return (b, a)
        else
            return (a, b)
    end
end

function constructFromFile(fname)
    f(x) = parse(Int, x)
    fc60 = open(fname, "r")
    lines = readlines(fc60)
    V = f(lines[1])
    E = Int(V * 3 / 2)
    # edges = list()
    edgesInd = Dict()
    edgeCount = 1
    edgeNneighbor = [[] for i in 1:E]
    vNeighbor = [[] for i in 1:V]
    for i in 3:length(lines)
        vs = zeros(Int, 4)
        r, c, x, y, z, vs[1], vs[2], vs[3], vs[4] = @scanf(lines[i], " %c %lf %lf %lf %d %d %d %d", Char, Float64, Float64, Float64, Int, Int, Int, Int)
        vNeighbor[vs[1]] = vs[2:4]
        for i in 1:3
            if vs[1+i] > vs[1]
                edgesInd[(vs[1], vs[1+i])] = edgeCount
                edgeCount += 1
            end
        end
        edges3 = [edgesInd[ordered(vs[1], vs[1+i])] for i in 1:3]
        for i in 1:3
            append!(edgeNneighbor[edges3[i]], [edges3[(i%3)+1], edges3[((i+1)%3)+1]])
        end
    end
    close(fc60)
    return V, E, vNeighbor, edgesInd, edgeNneighbor
end

function ising_graph_output(fin, fout)
    vNum, eNum, vN, eI, eN = constructFromFile(fin);
    outfile = open(fout, "w")
    edgeUsed = Set()
    s = string(eNum) * " " * string(eNum*2) * "\n"
    write(outfile, s)
    for i in 1:eNum
        eN[i] = eN[i] .- 1
        for j in 1:length(eN[i])
            e = ordered(i-1, eN[i][j])
            if ! (e in edgeUsed)
                s = string(i-1) * " " * string(eN[i][j]) * "\n"
                write(outfile, s)
                push!(edgeUsed, e)
            end
        end
    end
    close(outfile)
end;

function constructSkewMat(fname)
    infile = open(fname, "r")
    lines = readlines(infile)
    f(x) = parse(Int, x)
    V, E = f.(split(lines[1], " "))
    mat = zeros(BigInt, V, V)
    for i in 2:length(lines)
        data = f.(split(lines[i], " "))
        v1, v2 = ordered(data[1], data[2]) .+ 1
        sign = 2*data[3] - 1
        val = data[4] + 1
        mat[v1, v2] = sign*val
        mat[v2, v1] = -sign*val
    end
    close(infile)
    return mat
end;

function linearsys(m, n)
    cSwap(x) = CartesianIndex(x.I[2], x.I[1])
    A = zeros(BigInt, n, n)
    b = zeros(BigInt, n)
    c = findall(m .== 2)
    a0 = 0
    for i in 0:n
        for v in c
            m[v] = i
            m[cSwap(v)] = -i
        end
        if i == 0
            a0 = pfaffian(m)
            println(a0)
        else
            b[i] = pfaffian(m) - a0
            A[i, :] = [BigInt(i)^j for j in 1:n]
        end
    end
    return A, b
end;

function m2replace(m, val)
    cSwap(x) = CartesianIndex(x.I[2], x.I[1])
    c = findall(m .== 2)
    q = BigFloat.(copy(m))
    for v in c
        q[v] = val
        q[cSwap(v)] = -val
    end
    q
end;

In [3]:
# V = 30
# E = 45
ising_graph_output("fullerene_xyz/C180-0.xyz", "temp.in")

In [4]:
inname = "temp.in"
p = pipeline(`./build/main $inname`; stdout="1.out")
@time run(p)

 12.202176 seconds (28.03 k allocations: 1.911 MiB, 0.43% compilation time)


Process(`./build/main temp.in`, ProcessExited(0))

In [5]:
mat = constructSkewMat("1.out");
size(mat)

(2640, 2640)

## pfaffian and partition function
for $t = \exp(-\beta J)$, the pfaffian here can be understood as

$$ 2 * \text{Pf}(\text{mat}) = g \times t^{N} + O(t^{N+1})$$

where g is the G.S. degeneracy, and $N$ is the number of unsatisfied edges

In [6]:
R = Nemo.ArbField(1024)
RR = Nemo.RealField()
t = RR(1e-50)
@time pf = Nemo.sqrt( Nemo.det( Nemo.matrix(R, m2replace(mat, t)) ) ) * 2

219.013501 seconds (22.52 M allocations: 13.753 GiB, 0.97% gc time, 0.17% compilation time)


[6.5365996325895612112846672604590519064166445672505885483027430091230558082541945944915814327051436983278423911702353295084736207590805385533132915404600221422489802964515636975148576050758108154301656954e-8942 +/- 1.59e-9145]

since the numer of unsatisfied edges = 180, 

180*50 - 8942 = 58

therefore g = 6.5366e58

In [7]:
# # may also use BigFloat, but it seems that Arb is more efficient
# setprecision(BigFloat, 512)
# t = BigFloat(1e-50)
# @time pf = sqrt(det( BigFloat.(m2replace(mat, t))))*2

## From genericTensorNetwork, for comparison purpose

In [8]:
function constructGraph(fname)
    infile = open(fname, "r")
    lines = readlines(infile)
    f(x) = parse(Int, x)
    V, E = f.(split(lines[1], " "))
    els = []
    for i in 2:length(lines)
        data = f.(split(lines[i], " "))
        v1, v2 = ordered(data[1], data[2]) .+ 1
        push!(els, (v1, v2))
    end
    close(infile)
    return SimpleGraph(Edge.(els))
end;

In [10]:
ising_graph_output("fullerene_xyz/C20-Ih.xyz", "temp.in")
g = constructGraph("temp.in")

{30, 60} undirected simple Int64 graph

In [11]:
problem = SpinGlass(g; J=fill(-1, ne(g)));

In [12]:
Emin = solve(problem, SizeMin())[]

-20.0ₜ

In [13]:
@time partition_function = solve(problem, GraphPolynomial())[]

 13.891610 seconds (23.12 M allocations: 1.789 GiB, 2.40% gc time, 98.66% compilation time)


LaurentPolynomial(3251250*x⁻²⁰ + 22644000*x⁻¹⁶ + 72487100*x⁻¹² + 144113280*x⁻⁸ + 202822170*x⁻⁴ + 216538624 + 181438800*x⁴ + 121326720*x⁸ + 65536580*x¹² + 28872000*x¹⁶ + 10468200*x²⁰ + 3181440*x²⁴ + 826980*x²⁸ + 188160*x³² + 38480*x³⁶ + 6528*x⁴⁰ + 1290*x⁴⁴ + 160*x⁴⁸ + 60*x⁵² + 2*x⁶⁰)